# Q1

In [1]:
demographics = [80 34; 60 44; 40 44; 20 24; 40 114; 40 64; 70 14; 50 44; 70 54; 70 64]

10×2 Array{Int64,2}:
 80   34
 60   44
 40   44
 20   24
 40  114
 40   64
 70   14
 50   44
 70   54
 70   64

In [12]:
using JuMP, Cbc
m = Model(solver = CbcSolver())
@variable(m, district[1:10, 1:5], Bin)
@variable(m, dem_majority[1:5],Bin)

#only one district per city
for i in 1:10
    @constraint(m, sum(district[i,:]) == 1)
end

#district size requirement
for i in 1:5
    @constraint(m, sum(district[:,i] .* (demographics * [1; 1])) >= 150)
    @constraint(m, sum(district[:,i] .* (demographics * [1; 1])) <= 250)
end

#define dem majority
for i in 1:5
    # z(dem - rep - 1) >= 0
    @constraint(m, (dot(district[:,i] , 
                demographics[:,2]) - dot(district[:,i] , demographics[:,1]) - 1) >= (1 -dem_majority[i]) * -1000)
end

#maximize majorities
@objective(m, Max, sum(dem_majority))

solve(m)



:Optimal

In [14]:
println("Blue aquires the best of gerrymander by having democratic majority in ", getobjectivevalue(m) , 
    " districts, which is achieved by," )

district_vals = getvalue(district)
for i in 1:10
    for j in 1:5
        if district_vals[i,j] == 1
            println("Giving city " , i, " to district ", j)
        end
    end
end

Blue aquires the best of gerrymander by having democratic majority in 3.0 districts, which is achieved by,
Giving city 1 to district 5
Giving city 2 to district 5
Giving city 3 to district 2
Giving city 4 to district 2
Giving city 5 to district 1
Giving city 6 to district 4
Giving city 7 to district 3
Giving city 8 to district 2
Giving city 9 to district 4


# Q2

In [15]:
#Define cost matrix
A = [0 11 7 13 11; 5 0 13 15 15; 13 15 0 23 11; 9 13 5 0 3; 3 7 7 7 0]

5×5 Array{Int64,2}:
  0  11   7  13  11
  5   0  13  15  15
 13  15   0  23  11
  9  13   5   0   3
  3   7   7   7   0

In [17]:
m = Model(solver = CbcSolver())

#transitions used
@variable(m, x[1:5,1:5], Bin)

#ordering
@variable(m, u[1:5] >= 1, Int)

#1 incoming and outgoin edge
# and no transitioning to yourself
for i in 1:5
    @constraint(m, sum(x[i,:]) == 1)
    @constraint(m, sum(x[:,i]) == 1)
    @constraint(m, x[i,i] == 0)
end

#keep ordering as indexes
@constraint(m, u .<= 5)

#ordering implies transitions
for i in 1:5
    for j in 2:5
        @constraint(m, u[i] - u[j] + 5*x[i,j] <= 5-1)
    end
end

#minimize cost of cleaning time. 
#The making time is constant regardless of ordering so it doesn't matter
@objective(m, Min, sum(x.* A))
solve(m)

:Optimal

In [19]:
println("The optimal strategy which only requires only ", getobjectivevalue(m),
    " minutes of cleaning has the paints done in this order")
println(getvalue(u))

The optimal strategy which only requires only 41.0 minutes of cleaning has the paints done in this order
[1.0,5.0,3.0,2.0,4.0]


# Q3
## a

In [31]:
m = Model(solver = CbcSolver())

#Chess board
@variable(m, board[1:8, 1:8], Bin)

#exactly 8 pieces
@constraint(m, sum(board) == 8)

#row col threaten constraint
for i in 1:8
    @constraint(m, sum(board[:,i]) <= 1)
    @constraint(m, sum(board[i,:]) <= 1)
end

#diagonal
for i in 1:8
    @constraint(m, sum(Diagonal(board[i:8,:])) <= 1)
    @constraint(m, sum(Diagonal(board[:,i:8])) <= 1)
    @constraint(m, sum(Diagonal(board[i:8,end:-1:1])) <= 1)
    @constraint(m, sum(Diagonal(board[:,i:-1:1])) <= 1)
end
solve(m)

:Optimal

In [32]:
println("Valid Board")
board_val = getvalue(board)
for i in 1:8
    for j in 1:8
        if board_val[i,j] == 1
            print("x ")
        else
            print(". ")
        end
    end
    println()
end

Valid Board
. . . . . . . x 
. x . . . . . . 
. . . x . . . . 
x . . . . . . . 
. . . . . . x . 
. . . . x . . . 
. . x . . . . . 
. . . . . x . . 


## b

In [35]:
m = Model(solver = CbcSolver())

#Chess board
@variable(m, board[1:8, 1:8], Bin)

#exactly 8 pieces
@constraint(m, sum(board) == 8)

#row col threaten constraint
for i in 1:8
    @constraint(m, sum(board[:,i]) <= 1)
    @constraint(m, sum(board[i,:]) <= 1)
end

#diagonal
for i in 1:8
    @constraint(m, sum(Diagonal(board[i:8,:])) <= 1)
    @constraint(m, sum(Diagonal(board[:,i:8])) <= 1)
    @constraint(m, sum(Diagonal(board[i:8,end:-1:1])) <= 1)
    @constraint(m, sum(Diagonal(board[:,i:-1:1])) <= 1)
end

# symmetry
for i in 1:8
    for j in 1:8
        @constraint(m, board[i,j] == board[8 - i + 1, 8 - j + 1])
    end
end
solve(m)

:Optimal

In [36]:
println("Valid Board")
board_val = getvalue(board)
for i in 1:8
    for j in 1:8
        if board_val[i,j] == 1
            print("x ")
        else
            print(". ")
        end
    end
    println()
end

Valid Board
. . . x . . . . 
. . . . . x . . 
. . . . . . . x 
. x . . . . . . 
. . . . . . x . 
x . . . . . . . 
. . x . . . . . 
. . . . x . . . 


## c

In [38]:
m = Model(solver = CbcSolver())

#Chess board
@variable(m, board[1:8, 1:8], Bin)



#row col are covered
for i in 1:8
    @constraint(m, sum(board[:,i]) >= 1)
    @constraint(m, sum(board[i,:]) >= 1)
end

#diagonals are covered
for i in 1:8
    @constraint(m, sum(Diagonal(board[i:8,:])) >= 1)
    @constraint(m, sum(Diagonal(board[:,i:8])) >= 1)
    @constraint(m, sum(Diagonal(board[i:8,end:-1:1])) >= 1)
    @constraint(m, sum(Diagonal(board[:,i:-1:1])) >= 1)
end

@objective(m, Min, sum(board))


solve(m)

:Optimal

In [40]:
println("The smallest number of queens to threaten the whole board is ", getobjectivevalue(m))
println("And a board that accomplishes this is,")
board_val = getvalue(board)
for i in 1:8
    for j in 1:8
        if board_val[i,j] == 1
            print("x ")
        else
            print(". ")
        end
    end
    println()
end


The smallest number of queens to threaten the whole board is 16.0
And a board that accomplishes this is,
x . x . x . x x 
x . . . . . . . 
. . . . . . . x 
x . . . . . . . 
. . . . . . . x 
x . . . . . . . 
. . . . . . . x 
x x . x . x . x 


## d

In [42]:
m = Model(solver = CbcSolver())

#Chess board
@variable(m, board[1:8, 1:8], Bin)



#row col are covered
for i in 1:8
    @constraint(m, sum(board[:,i]) >= 1)
    @constraint(m, sum(board[i,:]) >= 1)
end

#diagonals are covered
for i in 1:8
    @constraint(m, sum(Diagonal(board[i:8,:])) >= 1)
    @constraint(m, sum(Diagonal(board[:,i:8])) >= 1)
    @constraint(m, sum(Diagonal(board[i:8,end:-1:1])) >= 1)
    @constraint(m, sum(Diagonal(board[:,i:-1:1])) >= 1)
end

# symmetry
for i in 1:8
    for j in 1:8
        @constraint(m, board[i,j] == board[8 - i + 1, 8 - j + 1])
    end
end

@objective(m, Min, sum(board))


solve(m)

:Optimal

In [43]:
println("The smallest number of queens to threaten the whole board is while having symmetry is ", getobjectivevalue(m))
println("And a board that accomplishes this is,")
board_val = getvalue(board)
for i in 1:8
    for j in 1:8
        if board_val[i,j] == 1
            print("x ")
        else
            print(". ")
        end
    end
    println()
end

The smallest number of queens to threaten the whole board is while having symmetry is 16.0
And a board that accomplishes this is,
x . . x . x x x 
x . . . . . . . 
x . . . . . . . 
. . . . . . . x 
x . . . . . . . 
. . . . . . . x 
. . . . . . . x 
x x x . x . . x 


This is the same number of pieces as without the symmetry constraint